##### Tutorial Credit
[BeuatifulSoup Tutorial - Zillow Web Scraping in Python](https://www.youtube.com/watch?v=dRcvJRmqFHQ&t=68s)

TODO:\
get bed and bath data\
possibly add in the floorplan values that are missing as NAN, more data = more better\
parse out special characters from csv, if a number is 250k turn the number into 250000\
\
ADD MORE LOCATION PARAMETERS, MORE DATA IS MORE BETTER, this will help with machine learning, long and lat will be help differentiate the price of a house in a specific area even if the model does't explicitely know what that area is

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from GrossText import KCHeader
from GrossText import OmahaHeader
from GrossText import StJoeHeader
#from GrossText import KCParams
import time
import csv
import random
import re

In [ ]:
class ZillowScraper():
    results = []

    def fetch(self, inurl, inparams, inHeader):
        response = requests.get(url=inurl, headers=inHeader, params=inparams)
        print(response.status_code)
        return response
    
    def parse(self, response):
        content = BeautifulSoup(response, 'lxml')
        deck = content.find('ul',{'class':'List-c11n-8-100-4__sc-1smrmqp-0 StyledSearchListWrapper-srp-8-100-4__sc-1ieen0c-0 cRHZNY KKlAT photo-cards photo-cards_extra-attribution'})
        for card in deck.contents:
            script = card.find('script',{'type': 'application/ld+json'})
            if script:
                script_json = json.loads(script.contents[0])
                    
                if 'floorSize' in script_json and 'value' in script_json['floorSize']:
                    #print(card.find('span', {'data-test': 'property-card-price', 'class': 'PropertyCardWrapper__StyledPriceLine-srp-8-100-4__sc-16e8gqd-1 dNnIRL'}).text)
                    floor_size_value = script_json['floorSize']['value']
                    if floor_size_value is not None:
                        if re.search('[,]',floor_size_value):
                            temp_floor_size_value = re.sub('[,]', '', floor_size_value)
                            floor_size_value = temp_floor_size_value
                        priceData = card.find('span', {'data-test': 'property-card-price', 'class': 'PropertyCardWrapper__StyledPriceLine-srp-8-100-4__sc-16e8gqd-1 dNnIRL'}).text
                        if re.search('[Kk]',priceData):
                            temp_priceData = re.sub('[Kk]', '000',priceData)
                            priceData = temp_priceData
                            continue
                        elif re.search('[+,$]',priceData):
                            temp_priceData = re.sub('[+,$]', '' , priceData)
                            priceData = temp_priceData
                            print(priceData)
                        self.results.append({
                            'latitude': script_json['geo']['latitude'],
                            'longitude': script_json['geo']['longitude'],
                            'floorSize': floor_size_value,
                            #'bedrooms' : 
                            #'bathrooms' :
                            'price': priceData,
                            #'price': card.find('span', {'data-test': 'property-card-price', 'class': 'PropertyCardWrapper__StyledPriceLine-srp-8-100-4__sc-16e8gqd-1 dNnIRL'}).text,
                            'url': script_json['url']
                        })
                else:
                    print("shits empty")
                    print(script_json['url'])


    def to_csv(self):
        if not self.results:
            print("No data to write to CSV.")
            return
        
        fieldnames = self.results[0].keys()  # Use keys from the first dictionary in results
        with open('zillow.csv', 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
            writer.writeheader()
            writer.writerows(self.results)  # Write all rows at once
            
    def run (self):
        header1 = KCHeader
        KCMOurl = "https://www.zillow.com/homes/Kansas-City,-MO_rb/"
        #1,21
        for page in range(1,21):
            KCMOparams = {'searchQueryState': '{"pagination":{"currentPage": %s},"isMapVisible":false,"mapBounds":{"west":-95.01517262499999,"east":-94.13626637499999,"south":36.563341336709954,"north":41.533945017316555},"usersSearchTerm":"Kansas City, MO","regionSelection":[{"regionId":18795,"regionType":6}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isListVisible":true,"mapZoom":8}' %page}
            KCMOres = self.fetch(KCMOurl, KCMOparams, header1)
            self.parse(KCMOres.text)
            randomNum = random.randint(30,60)
            time.sleep(randomNum)
    def run2 (self):
        header2 = StJoeHeader
        StJoeMOurl = "https://www.zillow.com/homes/saint-joseph,-MO_rb/"
        #1,6
        for page in range (1,6):
            StJoeMOParams = {'searchQueryState': '{"pagination":{"currentPage": %s},"isMapVisible":true,"mapBounds":{"west":-95.1487722088861,"east":-94.52735802431579,"south":39.68412123060132,"north":39.754102210360905},"regionSelection":[{"regionId":40704,"regionType":6}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isEntirePlaceForRent":true,"isRoomForRent":false,"isListVisible":true,"mapZoom":12}' %page}
            StJoeMOres = self.fetch(StJoeMOurl, StJoeMOParams, header2)
            self.parse(StJoeMOres.text)
            randomNum = random.randint(30,60)
            time.sleep(randomNum)
    def run3 (self):
        OmahaNEurl = "https://www.zillow.com/homes/Omaha,-NE_rb/"
        header3 = OmahaHeader
        #1,21
        for page in range (1,21):
            OmahaNeParams = {'searchQueryState': '{"pagination":{"currentPage": %s},"isMapVisible":true,"mapBounds":{"west":-96.7099553935547,"east":-95.46987360644532,"south":41.02791181891396,"north":41.527423622028884},"usersSearchTerm":"Omaha, NE","regionSelection":[{"regionId":40152,"regionType":6}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isListVisible":true}' %page}
            OmahaNEres = self.fetch(OmahaNEurl, OmahaNeParams, header3)
            self.parse(OmahaNEres.text)
            randomNum = random.randint(21,34)
            time.sleep(randomNum)
    def run4 (self):
        header4 = KCHeader
        OverlandParkurl = 'https://www.zillow.com/homes/Overland-Park,-KS_rb/'
        #1,7
        for page in range (1,7):
            OverlandPakrParams = {'searchQueryState': '{"pagination":{"currentPage": %s},"isMapVisible":false,"mapBounds":{"west":-94.68704246728515,"east":-94.24140953271484,"south":38.965938755457806,"north":39.62009235770738},"mapZoom":11,"regionSelection":[{"regionId":40202,"regionType":6}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isEntirePlaceForRent":true,"isRoomForRent":false,"isListVisible":true}' %page}
            OverlandPakrres = self.fetch(OverlandParkurl, OverlandPakrParams, header4)
            self.parse(OverlandPakrres.text)
            randomNum = random.randint(21,34)
            time.sleep(randomNum)
    def run5 (self):
        header5 = KCHeader
        KCKSurl = 'https://www.zillow.com/homes/Kansas-City,-KS_rb/'
        #1,8
        for page in range (1,8):
            KCKSParams = {'searchQueryState': '{"pagination":{"currentPage": %s},"isMapVisible":true,"mapBounds":{"west":-95.67895551153367,"east":-93.89642377325242,"south":38.64494514451059,"north":39.63548678854779},"mapZoom":9,"regionSelection":[{"regionId":39191,"regionType":6}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isEntirePlaceForRent":true,"isRoomForRent":false,"isListVisible":true}' %page}
            KCKSres = self.fetch(KCKSurl, KCKSParams, header5)
            self.parse(KCKSres.text)
            randomNum = random.randint(21,34)
            time.sleep(randomNum)

In [ ]:
scraper = ZillowScraper()

In [ ]:
scraper.run()

In [ ]:
scraper.run2()

In [ ]:
scraper.run3()

In [ ]:
scraper.run4()

In [ ]:
scraper.run5()

In [ ]:
scraper.to_csv()

---
### Vizualizaion time
##### Hope it works
plan is to make 2 graphs. first graph is a simple linear graph that plots price against square footage.\
Second graph is longitude and latitude for x,y and for each point I would like it to change color based on price and change size base on square footage

In [ ]:
# Code to remove duplicates
# dont need to run this again
'''
import csv

def remove_duplicates(input_file, output_file):
    # Initialize a set to keep track of seen rows
    seen_rows = set()

    # Open input and output CSV files
    with open(input_file, 'r', newline='') as csv_input,  \
         open(output_file, 'w', newline='') as csv_output:

        # Create CSV reader and writer objects
        reader = csv.reader(csv_input)
        writer = csv.writer(csv_output)

        # Iterate through each row in the input CSV file
        for row in reader:
            # Convert the row to a tuple to make it hashable
            row_tuple = tuple(row)
            # Check if the row is not a duplicate
            if row_tuple not in seen_rows:
                # Write the non-duplicate row to the output CSV file
                writer.writerow(row)
                # Add the row to the set of seen rows
                seen_rows.add(row_tuple)

# Usage example:
input_file = "Zillow Scrape 4.10.24.csv"
output_file = 'output_unique.csv'
remove_duplicates(input_file, output_file)
print(f"Unique data written to {output_file}")
'''

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file =  "4.10.24 Duplicates Removed.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file)

# Extract the "longitude" and "latitude" columns
longitude = df['longitude']
latitude = df['latitude']

# Plot longitude vs. latitude
plt.figure(figsize=(8, 6))  # Set the figure size (width, height)

plt.scatter(longitude, latitude, alpha=0.5)  # Plot the scatter plot
plt.title('Scatter Plot of Longitude vs. Latitude')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()


In [ ]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file)

# Find the index of the row with the maximum floor size
max_floor_size_index = df['floorSize'].idxmax()
max_price_index = df['price'].idxmax()

#getting ride of extreme outliers, they make the graph ugly
df_modified = df.drop(index=max_floor_size_index)
df_modified = df.drop(index=max_price_index)


# Extract the "price" and "floorSize" columns from the modified DataFrame
price = df_modified['price']
floor_size = df_modified['floorSize']

# Plot a scatter plot without the max floor size item
plt.figure(figsize=(10, 6))  # Set the figure size (width, height)

plt.scatter(floor_size, price, alpha=0.5)  # Plot the scatter plot
plt.title('Scatter Plot of Price vs. Floor Size (Excluding Outliers)')
plt.xlabel('Floor Size')
plt.ylabel('Price')
plt.grid(True)
plt.show()


---
### Machine Learning Time

going to try a bunch of different models to see if any is better than the other

#### TODO:
Poly Regression\
Kmeans\
Neural Net

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#Initial data reading, cleaning, and splitting
cleanedFile = "4.10.24 Duplicates Removed.csv"
DF = pd.read_csv(cleanedFile)

DF.dropna(axis='index', how='any', inplace=True)

train_set, test_set = train_test_split(DF, test_size=0.3, random_state=0)
print(DF.keys)

X = DF[['latitude', 'longitude', 'floorSize']]
y = DF['price']

X_train = train_set[['latitude', 'longitude', 'floorSize']]
y_train = train_set['price']

X_test = test_set[['latitude', 'longitude', 'floorSize']]
y_test = test_set['price']

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from math import sqrt
from sklearn.metrics import mean_squared_error

LR = LinearRegression()
LR.fit(X_train, y_train)

y_predicted = LR.predict(X_train)
print(LR.score(X_train,y_train))
print(mean_squared_error(y_train,y_predicted))
print(sqrt(mean_squared_error(y_train,y_predicted)))

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix


DTCWholeSet = DecisionTreeClassifier()
DTCWholeSet.fit(X_train,y_train)

DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train)

y_predicted = DTC.predict(X_train)
matrix = confusion_matrix(y_train,y_predicted)
print(matrix)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy", accuracy_score(y_train,y_predicted))
print("Precision", precision_score(y_train,y_predicted, average='weighted'))
print("Sensitivity", recall_score(y_train,y_predicted, average='weighted'))
print("F1", f1_score(y_train, y_predicted, average='weighted'))

^ Classic decision tree overfitting, I didn't give it any bounds

Decision tree 5 fold validation

In [ ]:
# Your code here
from sklearn.model_selection import StratifiedKFold



spliter = StratifiedKFold(n_splits = 5, shuffle=True)


for train_indices, validate_indices in spliter.split(X, y):
    DTC = DecisionTreeClassifier()
    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    DTC.fit(X_train,y_train)
    X_validate = X.iloc[validate_indices]
    y_validate = y.iloc[validate_indices]
    y_predicted = DTC.predict(X_validate)
    matrix = confusion_matrix(y_validate, y_predicted)
    print(matrix)
    print("Accuracy", accuracy_score(y_validate,y_predicted))
    print("F1", f1_score(y_validate, y_predicted, average='weighted'))
    print()

## SVM Linear

In [ ]:
# Your code here
from sklearn.svm import SVC
SVM = SVC()


SVM.fit(X_train, y_train)
y_train_predict = SVM.predict(X_train)
matrixTrain = confusion_matrix(y_train, y_train_predict)
print(matrixTrain)
print("Accuracy", accuracy_score(y_train, y_train_predict))
print("F1", f1_score(y_train ,y_train_predict, average='weighted'))



y_test_predict = SVM.predict(X_test)
matrixTest = confusion_matrix(y_test, y_test_predict)
print(matrixTest)
print("Accuracy", accuracy_score(y_test, y_test_predict))
print("F1", f1_score(y_test, y_test_predict, average='weighted'))

SVM 5 fold validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

spliter = StratifiedKFold(n_splits = 5)

for train_indices, validate_indices in spliter.split(X, y):
    SVM = SVC() #New model on each fold
    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    SVM.fit(X_train,y_train)
    X_validate = X.iloc[validate_indices]
    y_validate = y.iloc[validate_indices]
    y_predicted = SVM.predict(X_validate)
    matrix = confusion_matrix(y_validate, y_predicted)
    print(matrix)
    print("Accuracy", accuracy_score(y_validate,y_predicted))
    print("F1", f1_score(y_validate, y_predicted, average='weighted'))
    print()

## RBF SVM

In [ ]:
from sklearn.svm import SVC
SVM = SVC(kernel = 'rbf')


SVM.fit(X_train, y_train)
y_train_predict = SVM.predict(X_train)
matrixTrain = confusion_matrix(y_train, y_train_predict)
print(matrixTrain)
print("Accuracy", accuracy_score(y_train, y_train_predict))
print("F1", f1_score(y_train ,y_train_predict, average='weighted'))

y_test_predict = SVM.predict(X_test)
matrixTest = confusion_matrix(y_test, y_test_predict)
print(matrixTest)
print("Accuracy", accuracy_score(y_test, y_test_predict))
print("F1", f1_score(y_test, y_test_predict, average='weighted'))

RBF SVM 5 fold validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

spliter = StratifiedKFold(n_splits = 5)

for train_indices, validate_indices in spliter.split(X, y):
    SVM = SVC(kernel='rbf') #New model on each fold
    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    SVM.fit(X_train,y_train)
    X_validate = X.iloc[validate_indices]
    y_validate = y.iloc[validate_indices]
    y_predicted = SVM.predict(X_validate)
    matrix = confusion_matrix(y_validate, y_predicted)
    print(matrix)
    print("Accuracy", accuracy_score(y_validate,y_predicted))
    print("F1", f1_score(y_validate, y_predicted, average='weighted'))
    print()

## Poly SVM

In [ ]:
from sklearn.svm import SVC
SVM = SVC(kernel='poly')

SVM.fit(X_train, y_train)
y_train_predict = SVM.predict(X_train)
matrixTrain = confusion_matrix(y_train, y_train_predict)
print(matrixTrain)
print("Accuracy", accuracy_score(y_train, y_train_predict))
print("F1", f1_score(y_train ,y_train_predict, average='weighted'))


y_test_predict = SVM.predict(X_test)
matrixTest = confusion_matrix(y_test, y_test_predict)
print(matrixTest)
print("Accuracy", accuracy_score(y_test, y_test_predict))
print("F1", f1_score(y_test, y_test_predict, average='weighted'))

Poly SVM 5 fold validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

spliter = StratifiedKFold(n_splits = 5)

for train_indices, validate_indices in spliter.split(X, y):
    SVM = SVC(kernel='poly') #New model on each fold
    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    SVM.fit(X_train,y_train)
    X_validate = X.iloc[validate_indices]
    y_validate = y.iloc[validate_indices]
    y_predicted = SVM.predict(X_validate)
    matrix = confusion_matrix(y_validate, y_predicted)
    print(matrix)
    print("Accuracy", accuracy_score(y_validate,y_predicted))
    print("F1", f1_score(y_validate, y_predicted, average='weighted'))
    print()

## Sigmoid SVM

In [ ]:
from sklearn.svm import SVC
SVM = SVC(kernel='sigmoid')

SVM.fit(X_train, y_train)
y_train_predict = SVM.predict(X_train)
matrixTrain = confusion_matrix(y_train, y_train_predict)
print(matrixTrain)
print("Accuracy", accuracy_score(y_train, y_train_predict))
print("F1", f1_score(y_train ,y_train_predict, average='weighted'))

y_test_predict = SVM.predict(X_test)
matrixTest = confusion_matrix(y_test, y_test_predict)
print(matrixTest)
print("Accuracy", accuracy_score(y_test, y_test_predict))
print("F1", f1_score(y_test, y_test_predict, average='weighted'))

Sigmoid SVM 5 fold validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

spliter = StratifiedKFold(n_splits = 5)

for train_indices, validate_indices in spliter.split(X, y):
    SVM = SVC(kernel='sigmoid') #New model on each fold
    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    SVM.fit(X_train,y_train)
    X_validate = X.iloc[validate_indices]
    y_validate = y.iloc[validate_indices]
    y_predicted = SVM.predict(X_validate)
    matrix = confusion_matrix(y_validate, y_predicted)
    print(matrix)
    print("Accuracy", accuracy_score(y_validate,y_predicted))
    print("F1", f1_score(y_validate, y_predicted, average='weighted'))
    print()

## Stochastic Gradient Descent

In [ ]:
# Your code here
from sklearn.linear_model import SGDClassifier

SGD = SGDClassifier()

SGD.fit(X_train, y_train)
y_train_predict = SGD.predict(X_train)
matrixTrain = confusion_matrix(y_train, y_train_predict)
print(matrixTrain)
print("Accuracy", accuracy_score(y_train, y_train_predict))
print("F1", f1_score(y_train ,y_train_predict, average='weighted'))

y_test_predict = SGD.predict(X_test)
matrixTest = confusion_matrix(y_test, y_test_predict)
print(matrixTest)
print("Accuracy", accuracy_score(y_test, y_test_predict))
print("F1", f1_score(y_test, y_test_predict, average='weighted'))

Stochastic Gradient Descent 5 fold validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier

spliter = StratifiedKFold(n_splits = 5)

for train_indices, validate_indices in spliter.split(X, y):
    SGD = SGDClassifier()
    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    SGD.fit(X_train,y_train)
    X_validate = X.iloc[validate_indices]
    y_validate = y.iloc[validate_indices]
    y_predicted = SGD.predict(X_validate)
    matrix = confusion_matrix(y_validate, y_predicted)
    print(matrix)
    print("Accuracy", accuracy_score(y_validate,y_predicted))
    print("F1", f1_score(y_validate, y_predicted, average='weighted'))
    print()

## Random Forest

In [ ]:
# Your code here
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier()


RFC.fit(X_train, y_train)
y_train_predict = RFC.predict(X_train)
matrixTrain = confusion_matrix(y_train, y_train_predict)
print(matrixTrain)
print("Accuracy", accuracy_score(y_train, y_train_predict))
print("F1", f1_score(y_train ,y_train_predict, average='weighted'))

y_test_predict = RFC.predict(X_test)
matrixTest = confusion_matrix(y_test, y_test_predict)
print(matrixTest)
print("Accuracy", accuracy_score(y_test, y_test_predict))
print("F1", f1_score(y_test, y_test_predict, average='weighted'))

Random Forest 5 fold validation

In [ ]:

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier


spliter = StratifiedKFold(n_splits = 5)

for train_indices, validate_indices in spliter.split(X, y):
    RFC = RandomForestClassifier()
    X_train = X.iloc[train_indices]
    y_train = y.iloc[train_indices]
    RFC.fit(X_train,y_train)
    X_validate = X.iloc[validate_indices]
    y_validate = y.iloc[validate_indices]
    y_predicted = RFC.predict(X_validate)
    matrix = confusion_matrix(y_validate, y_predicted)
    print(matrix)
    print("Accuracy", accuracy_score(y_validate,y_predicted))
    print("F1", f1_score(y_validate, y_predicted, average='weighted'))
    print()